In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from vitrocal.datasets import catalog
from vitrocal.preprocessors import StandardPreprocessor
from vitrocal.detectors import DerivativeDetector, StandardExtractor
from vitrocal.analyzers import StandardAnalyzer

Load data

In [ ]:
datacatalog = catalog.DataCatalog()
df1 = datacatalog.load('e_green_dmso')
df2 = datacatalog.load('e_green')
df1.describe()

In [ ]:
df2.describe()

Define model using default parameters

In [ ]:
preprocessor = StandardPreprocessor(
    frames_per_second=1/2.5, 
    bleach_period=60,
    filter_frequency=.15, 
    baseline_threshold=10, 
    window_size=60)

extractor = StandardExtractor(
    window = (3, 30), # this is in seconds
    frames_per_second=1/2.5, 
    threshold=20)

analyzer = StandardAnalyzer(
    upper_decay_bound=.8,
    lower_decay_bound=.2
)

def pipeline(data, preprocessor, extractor, analyzer):
    d_f = preprocessor.preprocess(data)
    events = extractor.detect_and_extract(d_f)
    res, avg_res = analyzer.analyze(events)
    return d_f, events, res, avg_res

In [ ]:
d_f_df1, events_df1, res_df1, avg_res_df1 = pipeline(df1, preprocessor, extractor, analyzer)
d_f_df2, events_df2, res_df2, avg_res_df2 = pipeline(df2, preprocessor, extractor, analyzer)

In [ ]:
arbitrary_events = [1, 18, 26, 35, 41, 58]
e_dmso_green = d_f_df1.iloc[:, arbitrary_events]
e_green = d_f_df2.iloc[:, arbitrary_events]

def make_signal_plot(df1, df2, arbitrary_events, maxy=800, miny=0):

    n_events = len(arbitrary_events)
    n_samples = len(df1)
    dy = (maxy - miny) * 0.7

    fig, ax = plt.subplots()

    ax.set_ylim(-dy, n_events * dy)
    ax.set_yticks(np.arange(n_events) * dy, labels=[])
    ax.set_xlabel('frame')

    for i, (col1, col2) in enumerate(zip(df1, df2)):
        y1 = df1[col1] + i*dy
        y2 = df2[col2] + i*dy

        ax.plot(y1, color='teal', label='E DMSO Green')
        ax.plot(y2, color='blue', label= 'E Green')


make_signal_plot(e_dmso_green, e_green, arbitrary_events)
plt.savefig("../../data/04_reporting/data_compare.pdf", bbox_inches='tight')



In [ ]:
len(e_dmso_green)

In [ ]:
def plt_avg_detected(events, title=''):
    combined = pd.Series()
    for roi, values in events.items():
        for sequence in values:
            tmp = pd.Series(sequence)
            combined = pd.concat([combined, tmp], axis=1).agg("mean", axis=1)

    plt.plot(combined.sort_index())
    plt.title(title)

In [ ]:
plt_avg_detected(events_df1, "E DMSO Green")

In [ ]:
plt_avg_detected(events_df2, "E Green")
